In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
mainurl = 'https://contentservice.kz/'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

In [3]:
productlinks = []

In [4]:
testlink = 'https://contentservice.kz/product/2135260000007-kz-syr-sluckiy-belorusskoe-zoloto-45-kg'

r = requests.get(testlink, headers = headers)
soup = BeautifulSoup(r.content, 'lxml')    
name = soup.find('div', class_='col-lg-9 col-md-8 col-sm-12').text.strip()
category = soup.find('ol', class_='breadcrumb').text.strip()
rawcategory = str(category) 
second = rawcategory.splitlines()[3]
first = rawcategory.splitlines()[2]
rawname = name.splitlines()[6]
rawname = rawname.strip()
name = rawname
producer = soup.find('ul', class_='list-unstyled list-lg-pad').text.strip()
producer = producer.partition('\n')[0]
country = soup.find('ul', class_='list-unstyled list-lg-pad')
rawcountry = str(country)
country = re.findall(r'title="(.+?)"', rawcountry)
country = str(country) 
country = re.sub('[\W_]+', '', country)
if len(country)<1:
    country='No Data'
if len(producer)<1:
    producer='No Data'
print(first, second, name, producer, country)

Молочные продукты и яйца Сыр Сыр 50% Белорусское золото Cлуцкий сыродельный комбинат кг ТМ No Brand BY


In [5]:
pageurl = 'https://contentservice.kz/yogurt-1/'

r = requests.get(pageurl, headers = headers)
soup = BeautifulSoup(r.content, 'lxml')    


def getdata(pageurl):
    r = requests.get(pageurl, headers = headers)
    soup = BeautifulSoup(r.content, 'lxml')    
    return soup

def getnextpage(soup):
    dis = soup.find('li', class_='disabled')
    dis = str(dis)
    page = soup.find('ul', class_='pagination category-pagination')
    page = str(page)
    if ("Вперед" in dis):
        return
    else:
        suburl = re.findall(r'"Вперед" href="(.+?)"><span', page)    
        suburl = str(suburl)
        suburl = suburl.replace('[', '')
        suburl = suburl.replace(']', '')
        suburl = suburl.replace("'", '')
        pageurl = 'https://contentservice.kz' + str(suburl)
        return pageurl
pagelinks = []
pagelinks.append(pageurl)
while True:
    soup = getdata(pageurl)
    pageurl = getnextpage(soup)
    if not pageurl:
        break
    pagelinks.append(pageurl)
    
print(pagelinks)

['https://contentservice.kz/yogurt-1/', 'https://contentservice.kz/yogurt-1/page2/', 'https://contentservice.kz/yogurt-1/page3/', 'https://contentservice.kz/yogurt-1/page4/', 'https://contentservice.kz/yogurt-1/page5/', 'https://contentservice.kz/yogurt-1/page6/', 'https://contentservice.kz/yogurt-1/page7/']


In [6]:
productlinks = []
for link in pagelinks:
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'lxml')
    productlist = soup.find_all('div', class_='catalog__grid-view__item products-slider__item col-sm-6 wrap wrap-sm-2 col-lg-4 wrap-lg-3')
    for product in productlist:
        for link in product.find_all('a', href = True):
            productlinks.append('https://contentservice.kz' + link['href'])
            
productlinks=set(productlinks)
print(len(productlinks))
print(productlinks)

168
{'https://contentservice.kz/product/4606779772405-kz-yogurt-pitevoy-epica-klubn-marak-25-290ml-pb', 'https://contentservice.kz/product/4870207313335-kz-yogurt-food-master-grecheskiy-klubnika-72-130gr-stak', 'https://contentservice.kz/product/4606779900808-kz-yogurt-ehrmann-ermigurt-yablokogrusha-75-100gr-stak', 'https://contentservice.kz/product/4690228028960-kz-yogurt-agusha-zasypay-ka-zel-yablmelissa-8mes-27-200gr-pb', 'https://contentservice.kz/product/4606779608308-kz-puding-ehrmann-ermigurt-ekstra-royal-chocolate-32-100gr', 'https://contentservice.kz/product/4870005353717-kz-bioyogurt-pitevoy-emil-chernosliv-so-zlakami-15-500ml-trx', 'https://contentservice.kz/product/4690228036163-kz-bioyogurt-bio-max-citrus-mikskinoasemena-lna-24-125gr-stak', 'https://contentservice.kz/product/4606779762208-kz-yogurt-epica-nat-klubn-48-130gr-stak', 'https://contentservice.kz/product/4605627009540-kz-produkt-yogurtnyy-campina-nezhnyy-legkiy-klubn-01-320gr-stak', 'https://contentservice.kz/pro

In [8]:
scrapped = []
for link in productlinks:
    r = requests.get(link, headers = headers)
    soup = BeautifulSoup(r.content, 'lxml')    
    name = soup.find('div', class_='col-lg-9 col-md-8 col-sm-12').text.strip()
    category = soup.find('ol', class_='breadcrumb').text.strip()
    rawcategory = str(category) 
    second = rawcategory.splitlines()[3]
    first = rawcategory.splitlines()[2]
    rawname = name.splitlines()[6]
    rawname = rawname.strip()
    name = rawname
    producer = soup.find('ul', class_='list-unstyled list-lg-pad').text.strip()
    producer = producer.partition('\n')[0]
    country = soup.find('ul', class_='list-unstyled list-lg-pad')
    rawcountry = str(country)
    country = re.findall(r'title="(.+?)"', rawcountry)
    country = str(country) 
    country = re.sub('[\W_]+', '', country)
    producturl = link
    if len(country)<1:
        country='No Data'
    if len(producer)<1:
        producer='No Data'
        
    scrap = {
        'Category' : first,
        'Subcategory' : second,
        'Title' : name, 
        'Brand' : producer,
        'Country' : country,
        'URL' : producturl
    }
    
    scrapped.append(scrap)
    
df=pd.DataFrame(scrapped)

In [9]:
df

,Category,Subcategory,Title,Brand,Country,URL
0,Молочные продукты и яйца,Йогурт,Йогурт 2.5% Клубника-Маракуйя Epica п/бут 290г,ТМ Epica,RU,https://contentservice.kz/product/460677977240...
1,Молочные продукты и яйца,Йогурт,Йогурт 7.2% греческий Клубника FoodMaster ст 130г,ТМ Food Master,KZ,https://contentservice.kz/product/487020731333...
2,Молочные продукты и яйца,Йогурт,Продукт йогуртный 7.5% сливочный Яблоко-груша ...,ТМ Ehrmann,RU,https://contentservice.kz/product/460677990080...
3,Молочные продукты и яйца,Йогурт,Йогурт 2.7% для детей от 8мес Зелёное яблоко-М...,ТМ Агуша,RU,https://contentservice.kz/product/469022802896...
4,Молочные продукты и яйца,Йогурт,Пудинг 3.2% молочный ультрапастеризованный Шок...,ТМ Ehrmann,RU,https://contentservice.kz/product/460677960830...
...,...,...,...,...,...,...
163,Молочные продукты и яйца,Йогурт,Продукт йогуртный 2.5% пастеризованный фруктов...,ТМ Чудо,RU,https://contentservice.kz/product/469022800875...
164,Молочные продукты и яйца,Йогурт,Биойогурт 3.2% питьевой ванильный Био-С Имун+ ...,ТМ Food Master,KZ,https://contentservice.kz/product/487005500428...
165,Молочные продукты и яйца,Йогурт,Напиток кисломолочный 2% сладкий Снежок Food M...,ТМ Food Master,KZ,https://contentservice.kz/product/487020731288...
166,Молочные продукты и яйца,Йогурт,Продукт йогуртный 0.3% пастеризованный Киви-кр...,ТМ Ehrmann,RU,https://contentservice.kz/product/460701427022...


In [78]:
subcategories = []
testlink = 'https://contentservice.kz/catalog/'
r = requests.get(testlink, headers = headers)
soup = BeautifulSoup(r.content, 'lxml')    
list1 = soup.find('div', class_='col-xs-12 col-md-4 wrap-every-3 gategories-lists__item')
filter = list1.find_all('a', href = True)
filter = [ x for x in filter if len(x)<3 ]
filter


[<a href="/yogurt-1/">Йогурт <span class="cat-label">168</span></a>,
 <a href="/kefir-13/">Кефир <span class="cat-label">54</span></a>,
 <a href="/maslo-slivochnoe-19/">Масло сливочное <span class="cat-label">47</span></a>,
 <a href="/moloko-38/">Молоко <span class="cat-label">99</span></a>,
 <a href="/ryazhenka-6/">Ряженка <span class="cat-label">3</span></a>,
 <a href="/sguschenka-3/">Сгущенка <span class="cat-label">27</span></a>,
 <a href="/slivki-22/">Сливки <span class="cat-label">17</span></a>,
 <a href="/smetana-39/">Сметана <span class="cat-label">38</span></a>,
 <a href="/syr-7/">Сыр <span class="cat-label">252</span></a>,
 <a href="/tvorog-21/">Творог <span class="cat-label">43</span></a>,
 <a href="/tvorozhnye-deserty-3/">Творожные десерты <span class="cat-label">107</span></a>,
 <a href="/yayca-1/">Яйца <span class="cat-label">45</span></a>,
 <a href="/vafli-kz/">Вафли <span class="cat-label">62</span></a>,
 <a href="/vypechka-5/">Выпечка <span class="cat-label">3</span></

In [79]:
subcategories = []
testlink = 'https://contentservice.kz/catalog/'
r = requests.get(testlink, headers = headers)
soup = BeautifulSoup(r.content, 'lxml')    
list1 = soup.find('div', class_='col-xs-12 col-md-4 wrap-every-3 gategories-lists__item')
for link in filter:
    subcategories.append('https://contentservice.kz' + link['href'])
    

In [80]:
subcategories

['https://contentservice.kz/yogurt-1/',
 'https://contentservice.kz/kefir-13/',
 'https://contentservice.kz/maslo-slivochnoe-19/',
 'https://contentservice.kz/moloko-38/',
 'https://contentservice.kz/ryazhenka-6/',
 'https://contentservice.kz/sguschenka-3/',
 'https://contentservice.kz/slivki-22/',
 'https://contentservice.kz/smetana-39/',
 'https://contentservice.kz/syr-7/',
 'https://contentservice.kz/tvorog-21/',
 'https://contentservice.kz/tvorozhnye-deserty-3/',
 'https://contentservice.kz/yayca-1/',
 'https://contentservice.kz/vafli-kz/',
 'https://contentservice.kz/vypechka-5/',
 'https://contentservice.kz/korzhi-6/',
 'https://contentservice.kz/pechene-kz/',
 'https://contentservice.kz/pirozhnoe-4/',
 'https://contentservice.kz/pryaniki-10/',
 'https://contentservice.kz/suhariki-kz/',
 'https://contentservice.kz/sushki-5/',
 'https://contentservice.kz/torty-12/',
 'https://contentservice.kz/hleb-20/',
 'https://contentservice.kz/hlebcy-1/',
 'https://contentservice.kz/varene-7/

In [ ]:
scrapped = []
for link in productlinks:
    r = requests.get(link, headers = headers)
    soup = BeautifulSoup(r.content, 'lxml')    
    name = soup.find('div', class_='col-lg-9 col-md-8 col-sm-12').text.strip()
    category = soup.find('ol', class_='breadcrumb').text.strip()
    rawcategory = str(category) 
    second = rawcategory.splitlines()[3]
    first = rawcategory.splitlines()[2]
    rawname = name.splitlines()[6]
    rawname = rawname.strip()
    name = rawname
    producer = soup.find('ul', class_='list-unstyled list-lg-pad').text.strip()
    producer = producer.partition('\n')[0]
    country = soup.find('ul', class_='list-unstyled list-lg-pad')
    rawcountry = str(country)
    country = re.findall(r'title="(.+?)"', rawcountry)
    country = str(country) 
    country = re.sub('[\W_]+', '', country)
    producturl = link
    if len(country)<1:
        country='No Data'
    if len(producer)<1:
        producer='No Data'
        
    scrap = {
        'Category' : first,
        'Subcategory' : second,
        'Title' : name, 
        'Brand' : producer,
        'Country' : country,
        'URL' : producturl
    }
    
    scrapped.append(scrap)
    
df=pd.DataFrame(scrapped)

In [81]:
productlinks

{'https://contentservice.kz/product/4600605013267-kz-napitok-pitevoy-danone-actimel-klubnika-100ml-pb',
 'https://contentservice.kz/product/4600605015636-kz-napitok-pitevoy-danone-actimel-chernikaezhevika-100ml-pb',
 'https://contentservice.kz/product/4600605019696-kz-bioprodukt-danone-aktivia-tvorozhno-yogurtnyy-inzhirkuragachernosliv-42-130gr-stak',
 'https://contentservice.kz/product/4600605021088-kz-napitok-pitevoy-danone-actimel-kiviklubn-100ml-pb',
 'https://contentservice.kz/product/4600605021736-kz-napitok-pitevoy-danone-actimel-klubn-banan-det-100ml-pb',
 'https://contentservice.kz/product/4600605021804-kz-bioyogurt-danone-aktivia-termostatnaya-s-chernikoy-27-170gr-stak',
 'https://contentservice.kz/product/4600605021842-kz-bioyogurt-danone-aktivia-termostatnaya-35-170gr-stak',
 'https://contentservice.kz/product/4600605022962-kz-kokteyl-yogurtnyy-danissimo-shakego-klubn-morozhenoe-52-260ml-stak',
 'https://contentservice.kz/product/4600605023396-kz-napitok-pitevoy-danone-acti

In [91]:
productlinks = []
pagelinks = []
scrapped = []

In [ ]:
for subcategory in subcategories: 
    pageurl = subcategory

    r = requests.get(pageurl, headers = headers)
    soup = BeautifulSoup(r.content, 'lxml')    


    def getdata(pageurl):
        r = requests.get(pageurl, headers = headers)
        soup = BeautifulSoup(r.content, 'lxml')    
        return soup

    def getnextpage(soup):
        dis = soup.find('li', class_='disabled')
        dis = str(dis)
        page = soup.find('ul', class_='pagination category-pagination')
        page = str(page)
        if ("Вперед" in dis):
            return
        else:
            suburl = re.findall(r'"Вперед" href="(.+?)"><span', page)    
            suburl = str(suburl)
            suburl = suburl.replace('[', '')
            suburl = suburl.replace(']', '')
            suburl = suburl.replace("'", '')
            pageurl = 'https://contentservice.kz' + str(suburl)
            return pageurl
    
    pagelinks.append(pageurl)
    while True:
        soup = getdata(pageurl)
        pageurl = getnextpage(soup)
        if not pageurl:
            break
        pagelinks.append(pageurl)
        
    for link in pagelinks:
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'lxml')
        productlist = soup.find_all('div', class_='catalog__grid-view__item products-slider__item col-sm-6 wrap wrap-sm-2 col-lg-4 wrap-lg-3')
        for product in productlist:
            for link in product.find_all('a', href = True):
                productlinks.append('https://contentservice.kz' + link['href'])

    productlinks=set(productlinks)

        for link in productlinks:
            r = requests.get(link, headers = headers)
            soup = BeautifulSoup(r.content, 'lxml')    
            name = soup.find('div', class_='col-lg-9 col-md-8 col-sm-12').text.strip()
            category = soup.find('ol', class_='breadcrumb').text.strip()
            rawcategory = str(category) 
            second = rawcategory.splitlines()[3]
            first = rawcategory.splitlines()[2]
            rawname = name.splitlines()[6]
            rawname = rawname.strip()
            name = rawname
            producer = soup.find('ul', class_='list-unstyled list-lg-pad').text.strip()
            producer = producer.partition('\n')[0]
            country = soup.find('ul', class_='list-unstyled list-lg-pad')
            rawcountry = str(country)
            country = re.findall(r'title="(.+?)"', rawcountry)
            country = str(country) 
            country = re.sub('[\W_]+', '', country)
            producturl = link
            if len(country)<1:
                country='No Data'
            if len(producer)<1:
                producer='No Data'

            scrap = {
                'Category' : first,
                'Subcategory' : second,
                'Title' : name, 
                'Brand' : producer,
                'Country' : country,
                'URL' : producturl
            }

            scrapped.append(scrap)
    


In [94]:
df

,Category,Subcategory,Title,Brand,Country,URL
0,Молочные продукты и яйца,Йогурт,Йогурт 2.5% Клубника-Маракуйя Epica п/бут 290г,ТМ Epica,RU,https://contentservice.kz/product/460677977240...
1,Молочные продукты и яйца,Йогурт,Йогурт 7.2% греческий Клубника FoodMaster ст 130г,ТМ Food Master,KZ,https://contentservice.kz/product/487020731333...
2,Молочные продукты и яйца,Йогурт,Продукт йогуртный 7.5% сливочный Яблоко-груша ...,ТМ Ehrmann,RU,https://contentservice.kz/product/460677990080...
3,Молочные продукты и яйца,Йогурт,Йогурт 2.7% для детей от 8мес Зелёное яблоко-М...,ТМ Агуша,RU,https://contentservice.kz/product/469022802896...
4,Молочные продукты и яйца,Йогурт,Пудинг 3.2% молочный ультрапастеризованный Шок...,ТМ Ehrmann,RU,https://contentservice.kz/product/460677960830...
...,...,...,...,...,...,...
163,Молочные продукты и яйца,Йогурт,Продукт йогуртный 2.5% пастеризованный фруктов...,ТМ Чудо,RU,https://contentservice.kz/product/469022800875...
164,Молочные продукты и яйца,Йогурт,Биойогурт 3.2% питьевой ванильный Био-С Имун+ ...,ТМ Food Master,KZ,https://contentservice.kz/product/487005500428...
165,Молочные продукты и яйца,Йогурт,Напиток кисломолочный 2% сладкий Снежок Food M...,ТМ Food Master,KZ,https://contentservice.kz/product/487020731288...
166,Молочные продукты и яйца,Йогурт,Продукт йогуртный 0.3% пастеризованный Киви-кр...,ТМ Ehrmann,RU,https://contentservice.kz/product/460701427022...


In [95]:
import seaborn as sns

In [101]:
df.to_csv (r'C:\Users\Ali Shalbayev\Desktop\Ali_Shalbayev_BDA1902_HW4\output.csv', index = False, header=True)